<a href="https://colab.research.google.com/github/gladcolor/tweet_translator/blob/master/tweet_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Multi-language translation.

Please refer to [Beyond English-Centric Multilingual Machine Translation](https://arxiv.org/abs/2010.11125) and [Hugginface](https://huggingface.co/transformers/model_doc/m2m_100.html#overview).

In [4]:
! pip install sentencepiece
! pip install transformers==4.6.0

In [6]:
import os
import pandas as pd

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [7]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to("cuda:0")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

In [24]:
def trans(raw_text, out_lang = 'en'):
    encoded_text = tokenizer(raw_text, return_tensors="pt").to("cuda:0")
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(out_lang))
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return result

def main():

    in_lang = 'en'
    out_lang = 'zh'
    tokenizer.src_lang = in_lang
    tweets_csv = r"https://raw.githubusercontent.com/gladcolor/tweet_translator/master/tweets_samples.zip"
    
    tweets_df = pd.read_csv(tweets_csv, encoding='ISO-8859-1')
    tweets_df = tweets_df.sample(10)
    for idx, row in tweets_df.iterrows():
        raw_text = row["text"]
        result = trans(raw_text, out_lang=out_lang)
        print("Raw text:       ", raw_text)        
        print("Translated text:", result[0])
        print()


    in_lang = 'zh'
    out_lang = 'en'
    tokenizer.src_lang = in_lang
    tweets_csv = r"https://raw.githubusercontent.com/gladcolor/tweet_translator/master/food_for_thought.zip"
    
    tweets_df = pd.read_csv(tweets_csv)
    # tweets_df = tweets_df.sample(10)
    for idx, row in tweets_df.iterrows():
        raw_text = row["text"]
        result = trans(raw_text, out_lang=out_lang)
        print("Raw text:       ", raw_text)        
        print("Translated text:", result[0])
        print()


main()

Raw text:        @TeresaHotchkis1 98 days since AZ jab directly caused brain tissue damage.13 weeks since I last worked. Dizziness and fuzziness constant.Prognosis MS and/or Dementia. My company is considering dismissal, poor recovery.Please help

https://t.co/kZJDRt3gpD

https://t.co/dROOQkx0EN
Translated text: @TeresaHotchkis1 98 天以来 AZ jab 直接导致大脑组织损伤.13 周以来我最后工作. 麻烦和麻烦不断. 预测 MS 和 / 或 痴呆症. 我的公司正在考虑解雇,不良恢复. 请帮助 https://t.co/kZJDRt3gpD https://t.co/dROOQkx0EN

Raw text:        Center: UPHC Vaishali Nagar MHA
District: Jaipur I
Pincode: 302021
Type: Free
Date: 01-06-2021
Min Age: 45
Vaccine: COVISHIELD
Dose1: 3
Dose2: 0

Telegram: https://t.co/cBSgxuLt65

#jaipur #rajasthan #covid19 #vaccine #slot #covishield #covaxin #SputnikV
[20:26]
Translated text: 中心: UPHC Vaishali Nagar MHA 区: Jaipur I Pincode: 302021 类型: 免费日期: 01-06-2021 年龄: 45 疫苗: COVISHIELD 剂量1: 3 剂量2: 0 电报: https://t.co/cBSgxuLt65 #jaipur #rajasthan #covid19 #vaccine #slot #covishield #covaxin #SputnikV [20:26]

Raw text:     